In [9]:
import sys
sys.path.append('..')
from src.core import *

In [10]:
bank1 = Bank()
bank2 = Bank()

In [11]:
# Поэтапное создание клиентов
client1 = Client(bank1, "Иван", "Иванов")
client1.address = "Москва"
client1.passport = "1234 567890"

client1Facade = ClientFacade(client1)
debit_1 = client1Facade.create_account('DebitAccount')

client2 = Client(bank2, "Петр", "Петров")
client2Facade = ClientFacade(client2)
debit_2 = client2Facade.create_account('DebitAccount')

In [12]:
# Внесение наличных

client1Facade.deposit(debit_1.id, 1000)
debit_1.balance

1000

In [13]:
# Перевод

Transaction(debit_1, debit_2, 500).perform()
debit_1.balance, debit_2.balance

(500, 500)

In [14]:
# Альтернативно, клиенты могут через фасад переводить по id счёта получателя

debit_2.id

UUID('1558b781-e8a6-48cb-871c-82cf5598e921')

In [15]:
client1Facade.transfer(debit_1.id, debit_2.id, 500, bank2)
debit_1.balance, debit_2.balance

(0, 1000)

In [16]:
# Если просто попросить Петрова вернуть деньги, то он не сможет, т.к. у него нет паспорта

Transaction(debit_2, debit_1, 500).perform()

Error: Client doesn't have passport or address

In [17]:
# Но поможет механизм отмены транзакции
transactions = debit_1.history.see()
transactions

[*aa2b -> *3fad: 1000 at 2023-04-14 23:40:37.481703,
 *8e92 -> *3fad: -500 at 2023-04-14 23:40:38.446041,
 *8e92 -> *3fad: -500 at 2023-04-14 23:40:40.392026]

In [18]:
transactions[-1].cancel()
transactions[-2].cancel()
debit_1.balance, debit_2.balance

(1000, 0)

In [19]:
# История с отрицательными транзакциями выглядит немного странно,
# но идея в том, что сумма всех чисел равна конечному балансу
debit_1.history.see()

[*aa2b -> *3fad: 1000 at 2023-04-14 23:40:37.481703,
 *8e92 -> *3fad: -500 at 2023-04-14 23:40:38.446041,
 *8e92 -> *3fad: -500 at 2023-04-14 23:40:40.392026,
 *8e92 -> *3fad: 500 at 2023-04-14 23:40:43.173764,
 *8e92 -> *3fad: 500 at 2023-04-14 23:40:43.173993]